In [ ]:
import numpy as np
import pandas as pd

import ast
import random

In [ ]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="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",
)

In [ ]:
raw_link = pd.read_csv('../Data/postgenerate_gpt.csv')
raw_tthc = pd.read_csv('../Data/postprocessed_tthc.csv', dtype={'maThuTuc': str})

In [ ]:
raw_link['TTHCLienQuan'] = raw_link['TTHCLienQuan'].apply(ast.literal_eval)

def getKnowledge(s):
     l = raw_tthc[raw_tthc['link'].isin(s)].drop(columns=['link']).reset_index(drop=True).T.to_dict().values()
     knowledge = ""
     for i in l:
          knowledge += str(i) + "\n"
     return knowledge

In [ ]:
template_df = pd.read_csv('../Template/template.csv', index_col=0)
config_df = pd.read_csv('../Template/config.csv', index_col=0)

In [ ]:
cols = ['link', 'Mã thủ tục hành chính liên quan', 'Số quyết định', 'Tên thủ tục', 'Cấp thực hiện', 'Loại thủ tục', 'Lĩnh vực', 
        'Trình tự thực hiện', 'Cách thức thực hiện', 'Thành phần hồ sơ', 'Đối tượng thực hiện', 'Cơ quan thực hiện', 
        'Cơ quan có thẩm quyền', 'Địa chỉ tiếp nhận hồ sơ', 'Cơ quan được ủy quyền', 'Cơ quan phối hợp', 'Kết quả thực hiện', 
        'Căn cứ pháp lý', 'Yêu cầu, điều kiện thực hiện', 'Từ khóa', 'Mô tả']

In [ ]:
raw_tthc.columns = cols

In [ ]:
pattern = []

pattern.append("Bạn đang cố gắng xác định xem câu trả lời có hiểu sai ngữ cảnh và mục đích của câu hỏi hay không.")
pattern.append("Bạn đang cố gắng xác định xem câu trả lời có sự mâu thuẫn về mặt thực tế so với kiến thức liên quan hay không. Một số thông tin trong câu trả lời có thể đã được bịa ra.")
pattern.append("Bạn đang cố gắng xác định xem câu trả lời có quá chung chung hoặc quá chi tiết so với mức độ cụ thể phù hợp để trả lời câu hỏi hay không.")
pattern.append("Bạn đang cố gắng xác định xem câu trả lời có thể được suy ra đúng từ kiến thức liên quan hay không.")

In [ ]:
evaluate_context = template_df['evaluate_context']['close_source']
print(evaluate_context)

In [ ]:
evaluate_template = template_df['evaluate_template']['close_source']
print(evaluate_template)

In [ ]:
model_name = "Claude_3.5_Haiku"

filename = f'{model_name}_evaluate.csv'

cols = ['link', 'cauTraLoiDung', 'danhGiaDung', 'cauTraLoiAoGiac', 'pattern', 'danhGiaAoGiac']

try:
    df = pd.read_csv(filename)
    if df.empty:
        print("CSV file is empty")
        df = pd.DataFrame(columns=cols)
except FileNotFoundError:
    print(f"File not found: {filename}")
    df = pd.DataFrame(columns=cols)
    df.to_csv(filename, index=False)
except pd.errors.EmptyDataError:
    print(f"File is completely empty: {filename}")
    df = pd.DataFrame(columns=cols)
    df.to_csv(filename, index=False)

for i in range(len(raw_link)):
    if raw_link['link'][i] in list(df['link'].values):
        continue

    p = raw_link['pattern'][i]
    question = raw_link['cauHoi'][i]
    right_answer = raw_link['cauTraLoi'][i]
    hallucinated_answer = raw_link['cauTraLoiAoGiac'][i]

    context_right = evaluate_context.format(pattern="")
    context_hallucinated = evaluate_context.format(pattern=pattern[p])

    knowledge = getKnowledge(raw_link['TTHCLienQuan'][i])
    prompt_right = evaluate_template.format(knowledge=knowledge, question=question, answer=hallucinated_answer)
    prompt_hallucinated = evaluate_template.format(knowledge=knowledge, question=question, answer=hallucinated_answer)

    print(prompt_right)

    try:
        completion = client.chat.completions.create(
        model="anthropic/claude-3-haiku:beta",
        messages=[
            {"role": "system", 
             "content": context_right
            },   
            {
            "role": "user",
            "content":prompt_right
            }
        ],
        temperature=0.7,
        max_tokens=512,
        top_p=0.9
        )
        output_right = str(completion)
        completion = client.chat.completions.create(
        model="anthropic/claude-3-haiku:beta",
        messages=[
            {"role": "system", 
             "content": context_hallucinated
            },   
            {
            "role": "user",
            "content":prompt_hallucinated
            }
        ],
        temperature=0.7,
        max_tokens=512,
        top_p=0.9
        )
        output_hallucinated = str(completion)
    except Exception as e:
        output_right = ""
        output_hallucinated = ""
        print(e)
        break

    results = [raw_link['link'][i], right_answer, output_right, hallucinated_answer, p, output_hallucinated]
    df.loc[i, cols] = results    
    df.to_csv(filename, encoding='utf-8-sig', index=False)

    print(f'Done of {len(raw_link)}: {i+1}', end='\r')